In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from IPython.display import HTML
from selenium import webdriver
import time as time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
# Instalation
s = Service((r'C:\Users\eli\Desktop\AllMyRepos\Web-Scraping-with-BeautifulSoup-and-Selenium\Project No.10\chromedriver.exe'))
chromeOptions = Options()
chromeOptions.headless = False
driver = webdriver.Chrome(service=s, options=chromeOptions)

In [3]:
# lists
titles_ = []
companies_ = []
ratings_ = []
locations_ = []
salaries_ = []
links_ = []

In [4]:
url = 'https://www.indeed.com/jobs?q=data+analyst&l=New+York%2C+NY&start=0&pp=gQAPAAABhIvz3qEAAAAB7kC5IwAnAQEBCFFbITFk9V4_HS90YSFZhUXFWI1BeZMmPL72AuEgB78FZWsAAAA&vjk=a44a90be68296825'
while True:
    # Get URL
    driver.get(url)
    time.sleep(3)
    
    # Soup Object
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Results
    results = soup.findAll('div', class_ = 'cardOutline')
    for result in results:
        try:  
            title = result.find('h2', class_ = 'jobTitle').text
            titles_.append(title)
        except:
            titles_.append(np.nan)

        try:
            company = result.find('span', class_ = 'companyName').text
            companies_.append(company)
        except:
            companies_.append(np.nan)

        try:
            rating = result.find('span', class_ = 'ratingNumber').text
            ratings_.append(rating)
        except:
            ratings_.append(np.nan)

        try:
            location = result.find('div', class_ = 'companyLocation').text
            locations_.append(location)
        except:
            locations_.append(location)

        salary = result.find(class_ = 'metadata salary-snippet-container')
        estimated_salary = result.find('div',class_ = 'metadata estimated-salary-container')
        if salary:
            salaries_.append(salary.text)
        elif estimated_salary:
            salaries_.append(estimated_salary.text)
        else:
            salaries_.append(np.nan)
        
        try:
            link = result.find(class_ ='jcs-JobTitle css-jspxzf eu4oa1w0')['href']
            links_.append('https://www.indeed.com/' + link)
        except:
            links.append(np.nan)
    
    next_url1 = soup.find('a', attrs = {'aria-label' : 'Next Page'})
    next_url2 = soup.find('a', attrs = {'aria-label' : 'Next'})
    if next_url1:
        url = 'https://www.indeed.com/' + next_url1.get('href')
    elif next_url2:
        url = 'https://www.indeed.com/' + next_url2.get('href')
    else:
        break

In [5]:
df_job_posts = pd.DataFrame({ 'Title' : titles_,'Company': companies_,'Rating': ratings_, 
                           'Location' : locations_, 'Salary' : salaries_,
                            'Link': links_})

In [6]:
df_job_posts.head(10)

,Title,Company,Rating,Location,Salary,Link
0,Data Analyst,Fox Corporation,4.0,"Hybrid remote in New York, NY",Estimated $73.4K - $93K a year,https://www.indeed.com//rc/clk?jk=a58ee3e72a6c...
1,Associate Analyst Reporting & Data Analytics,Madison Square Garden Entertainment,3.9,"New York, NY 10121 (Garment District area)",Estimated $64.7K - $81.9K a year,https://www.indeed.com//rc/clk?jk=51e4bd457dbd...
2,"Data Analyst, Product Operations",Riskified,NaN,"Hybrid remote in New York, NY 10001","$90,000 - $120,000 a year",https://www.indeed.com//rc/clk?jk=a44a90be6829...
3,Part-time Data Analyst I (ONSITE: 18-19 hours ...,Mount Sinai,3.7,"New York, NY","$58,661 - $74,250 a year",https://www.indeed.com//rc/clk?jk=e954125eddb7...
4,Data Analyst,Dow Jones,3.9,"New York, NY 10176 (Murray Hill area)","$40,000 - $160,000 a year",https://www.indeed.com//rc/clk?jk=4823d05ce00f...
5,Data Analyst,Tradeweb Markets LLC,3.7,"New York, NY 10001 (Chelsea area)",Estimated $91.9K - $116K a year,https://www.indeed.com//rc/clk?jk=73bd6d30f493...
6,Part- Time Data Analyst,Mercy College,3.9,"Dobbs Ferry, NY 10522",$40 an hour,https://www.indeed.com//rc/clk?jk=f11df3513be3...
7,"Data Analyst, CX Data, Forecasting & Systems",Away,3.0,"New York, NY","$97,000 - $110,000 a year",https://www.indeed.com//rc/clk?jk=5d0e3661f9a3...
8,Payment Integrity Analyst - Remote,EmblemHealth,3.5,"Remote in New York, NY","$65,000 - $75,000 a year",https://www.indeed.com//rc/clk?jk=ce6a3d9e77d3...
9,Data Analyst,S&P Global,3.9,"New York, NY","$74,067 - $128,019 a year",https://www.indeed.com//rc/clk?jk=10edf4b58fcc...


### Data Cleaning

In [7]:
df_job_posts.drop_duplicates(keep = 'first', inplace = True, ignore_index = False)

In [8]:
df_job_posts['Rating'] = pd.to_numeric(df_job_posts['Rating'], errors = 'coerce')
df_job_posts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 940 entries, 0 to 982
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     940 non-null    object 
 1   Company   940 non-null    object 
 2   Rating    730 non-null    float64
 3   Location  940 non-null    object 
 4   Salary    762 non-null    object 
 5   Link      940 non-null    object 
dtypes: float64(1), object(5)
memory usage: 51.4+ KB


In [9]:
HTML(df_job_posts.to_html(render_links = True, escape = False))

,Title,Company,Rating,Location,Salary,Link
0,Data Analyst,Fox Corporation,4.0,"Hybrid remote in New York, NY",Estimated $73.4K - $93K a year,https://www.indeed.com//rc/clk?jk=a58ee3e72a6cb854&fccid=61c19e057c54ec35&vjs=3
1,Associate Analyst Reporting & Data Analytics,Madison Square Garden Entertainment,3.9,"New York, NY 10121 (Garment District area)",Estimated $64.7K - $81.9K a year,https://www.indeed.com//rc/clk?jk=51e4bd457dbd7c20&fccid=4d2b33d40412d480&vjs=3
2,"Data Analyst, Product Operations",Riskified,NaN,"Hybrid remote in New York, NY 10001","$90,000 - $120,000 a year",https://www.indeed.com//rc/clk?jk=a44a90be68296825&fccid=777f6e070f6b7973&vjs=3
3,Part-time Data Analyst I (ONSITE: 18-19 hours per week) - Addiction Institute of Mount Sinai,Mount Sinai,3.7,"New York, NY","$58,661 - $74,250 a year",https://www.indeed.com//rc/clk?jk=e954125eddb7bbdf&fccid=c007936ceb766fe5&vjs=3
4,Data Analyst,Dow Jones,3.9,"New York, NY 10176 (Murray Hill area)","$40,000 - $160,000 a year",https://www.indeed.com//rc/clk?jk=4823d05ce00f9881&fccid=d2621f51dde471d3&vjs=3
5,Data Analyst,Tradeweb Markets LLC,3.7,"New York, NY 10001 (Chelsea area)",Estimated $91.9K - $116K a year,https://www.indeed.com//rc/clk?jk=73bd6d30f49347f5&fccid=737cbaa7a85af001&vjs=3
6,Part- Time Data Analyst,Mercy College,3.9,"Dobbs Ferry, NY 10522",$40 an hour,https://www.indeed.com//rc/clk?jk=f11df3513be35290&fccid=2e63343d9fed158d&vjs=3
7,"Data Analyst, CX Data, Forecasting & Systems",Away,3.0,"New York, NY","$97,000 - $110,000 a year",https://www.indeed.com//rc/clk?jk=5d0e3661f9a3d24b&fccid=a972580043b07190&vjs=3
8,Payment Integrity Analyst - Remote,EmblemHealth,3.5,"Remote in New York, NY","$65,000 - $75,000 a year",https://www.indeed.com//rc/clk?jk=ce6a3d9e77d38cb8&fccid=059d889e14e3a9b4&vjs=3
9,Data Analyst,S&P Global,3.9,"New York, NY","$74,067 - $128,019 a year",https://www.indeed.com//rc/clk?jk=10edf4b58fccd349&fccid=b716e44d2c6283e7&vjs=3


In [10]:
df_job_posts.to_excel('data_analyst_job_posts.xlsx', index = False)